In [1]:
import pandas as pd

df = pd.read_csv("food_rows.csv")
#df.head()

In [ ]:
## add other statistics
## look  for nan values or duplicated rows
df.describe()
df.isnull().sum()
df.duplicated().sum()

np.int64(0)

In [27]:
## change formatting for better embeddings

data = df.to_dict("records")

In [17]:
%pip install qdrant-client
%pip install sentence-transformers


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import os

HF_token = os.getenv("HF_TOKEN")
Gemini_api_key = os.getenv("GEMINI_API_KEY")

In [18]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

encoder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [20]:
qdrant = QdrantClient(":memory:")

In [25]:
qdrant.create_collection(
    collection_name="food_rows",
    vectors_config=models.VectorParams(size=encoder.get_sentence_embedding_dimension(), distance=models.Distance.COSINE)

)

ValueError: Collection food_rows already exists

In [40]:
qdrant.upload_points(
   collection_name="food_rows",
   points=[
      models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["name"]).tolist(),
            payload=doc
      )
      for idx, doc in enumerate(data)
   ]
)

In [ ]:
user_prompt=""

In [ ]:
hits = qdrant.search(
   collection_name="food_rows",
   query_vector=encoder.encode(user_prompt).tolist(),
   limit=5  # Return 5 closest points
)

for hit in hits:
   print(hit.payload, "score:", hit.score)

{'id': 478, 'recipeid': 557, 'name': 'Baked Breakfast Potatoes', 'recipeingredientquantities': '1 1/4 1 1 4 4 2 1', 'units': '["tablespoon","teaspoon","ounces","ounces","lb","cup"]', 'recipeingredientparts': 'black pepper nonfat sour cream sour cream low-fat sharp cheddar cheese sharp cheddar cheese Kraft Healthy Favorites Fat Free Cheddar cheddar cheese potatoes', 'calories': 149.3, 'fatcontent': 1.6, 'carbohydratecontent': 26.7, 'fibercontent': 2.1, 'proteincontent': 7.4, 'recipeinstructions': 'In a large bowl combine all ingredients except the corn flakes. Mix well and pour into a 9 X 13 inch casserole sprayed with non-stick cooking spray. Sprinkle crushed corn flakes on top. Bake at 350 degrees F or 45 to 60 minutes.'} score: 0.38160507554690803
{'id': 491, 'recipeid': 575, 'name': 'Baked Apple Slices', 'recipeingredientquantities': '12 8 1/2 1/2', 'units': '["tablespoons","cups","teaspoon"]', 'recipeingredientparts': 'apples butter sugar cinnamon', 'calories': 292.6, 'fatcontent':

/tmp/ipykernel_4255/1738407276.py:1: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = qdrant.search(


In [41]:
search_results = [hit.payload for hit in hits]

In [ ]:
## GEMINI